The dataset we will be working with contains information on various cars. For each car we have information about the technical aspects of the vehicle such as the motor's displacement, the weight of the car, the miles per gallon, and how fast the car accelerates. Using this information we will predict the origin of the vehicle, either North America, Europe, or Asia. We have three categories to choose from

The dataset is hosted by the University of California Irvine on their [machine learning repository](https://archive.ics.uci.edu/ml/datasets/Auto+MPG). As a side note, the UCI Machine Learning repository contains many small datasets which are useful when getting hands dirty with machine learning.

Data Folder contains a few different files. We'll be working with auto-mpg.data, which omits the 8 rows containing missing values for fuel efficiency (mpg column). We've converted this data into a CSV file named auto.csv

Here are the columns in the dataset:

* mpg -- Miles per gallon, Continuous.
* cylinders -- Number of cylinders in the motor, Integer, Ordinal, and Categorical.
* displacement -- Size of the motor, Continuous.
* horsepower -- Horsepower produced, Continuous.
* weight -- Weights of the car, Continuous.
* acceleration -- Acceleration, Continuous.
* year -- Year the car was built, Integer and Categorical.
* origin -- Integer and Categorical. 1: North America, 2: Europe, 3: Asia.

In [1]:
import pandas as pd
import numpy as np
cars = pd.read_csv("auto.csv")
unique_regions = cars["origin"].unique()
print(unique_regions)
cars.head()

[1 3 2]


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In many cases, like with this dataset, we'll have to create numeric representation of categorical values ourself. For this dataset, categorical variables exist in three columns, cylinders, year, and origin. The cylinders and year columns must be converted to numeric values so we can use them to predict label origin.  Even though the column year is a number, we’re going to treat them like categories. The year 71 is unlikely to relate to the year 70 in the same way those two numbers do numerically, but rather just as two different labels. In these instances, it is always safer to treat discrete values as categorical variables.

We must use **dummy variables** for columns containing categorical values. Whenever we have more than 2 categories, we need to create more columns to represent the categories. Since we have 5 different categories of cylinders, we could use 3, 4, 5, 6, and 8 to represent the different categories. We can split the column into separate binary columns:

* cyl_3 -- Does the car have 3 cylinders? 0 if False, 1 if True.
* cyl_4 -- Does the car have 4 cylinders? 0 if False, 1 if True.
* cyl_5 -- Does the car have 5 cylinders? 0 if False, 1 if True.
* cyl_6 -- Does the car have 6 cylinders? 0 if False, 1 if True.
* cyl_8 -- Does the car have 8 cylinders? 0 if False, 1 if True.

In [2]:
dummy_cylinders = pd.get_dummies(cars["cylinders"],prefix = "cyl")
dummy_years = pd.get_dummies(cars["year"], prefix = "year")

In [3]:
cars = pd.concat([cars, dummy_cylinders, dummy_years], axis = 1)

In [4]:
cars.drop(["cylinders","year"], axis = 1, inplace = True)
cars.head()

,mpg,displacement,horsepower,weight,acceleration,origin,cyl_3,cyl_4,cyl_5,cyl_6,...,year_73,year_74,year_75,year_76,year_77,year_78,year_79,year_80,year_81,year_82
0,18.0,307.0,130.0,3504.0,12.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15.0,350.0,165.0,3693.0,11.5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18.0,318.0,150.0,3436.0,11.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16.0,304.0,150.0,3433.0,12.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17.0,302.0,140.0,3449.0,10.5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


When we have 3 or more categories, we call the problem a **multiclass classification** problem. There are a few different methods of doing multiclass classification and in this project, we'll focus on the **one-versus-all** method.

The one-versus-all method is a technique where we choose a single category as the Positive case and group the rest of the categories as the False case. We're essentially splitting the problem into multiple binary classification problems. For each observation, the model will then output the probability of belonging to each category.

In [5]:
shuffled_rows = np.random.permutation(cars.index)

In [6]:
shuffled_cars = cars.iloc[shuffled_rows]
train = shuffled_cars.iloc[:int(shuffled_cars.shape[0]*.7)]
test = shuffled_cars.iloc[int(shuffled_cars.shape[0]*.7):]
print(len(train))
print(len(test))

274
118


In the one-vs-all approach, we're essentially converting an n-class (in our case n is 3) classification problem into n binary classification problems. For our case, we'll need to train 3 models:

* A model where all cars built in North America are considered Positive (1) and those built in Europe and Asia are considered Negative (0).
* A model where all cars built in Europe are considered Positive (1) and those built in North America and Asia are considered Negative (0).
* A model where all cars built in Asia are labeled Positive (1) and those built in North America and Europe are considered Negative (0).

Each of these models is a binary classification model that will return a probability between 0 and 1. When we apply this model on new data, a probability value will be returned from each model (3 total). For each observation, we choose the label corresponding to the model that predicted the highest probability.

We'll use the dummy variables we created from the cylinders and year columns to train 3 models using the LogisticRegression class from scikit-learn.

For each value in unique_origins, train a logistic regression model with the following parameters:

* X: Dataframe containing just the cylinder & year binary columns.
* y: list (or Series) of Boolean values:
 * True if observation's value for origin matches the current iterator variable.
 * False if observation's value for origin doesn't match the current iterator variable.

In [11]:
from sklearn.linear_model import LogisticRegression

unique_origins = cars["origin"].unique()
unique_origins.sort()

In [12]:
models = {}

features = [c for c in train.columns if c.startswith("cyl") or c.startswith("year")]

for origin in unique_origins:
    model = LogisticRegression()
    model.fit(train[features], train["origin"] == origin)
    models[origin] = model
    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Now that we have a model for each category, we can run our test dataset through the models and evaluate how well they performed.

In [19]:
# For each origin value from unique_origins:
# Use the LogisticRegression predict_proba function to return the 3 lists of predicted probabilities for the test set 

testing_probs = pd.DataFrame(columns=unique_origins)

for origin in unique_origins:
    testing_probs[origin] = models[origin].predict_proba(test[features])[:,1]
    

In [23]:
testing_probs.head()

,1,2,3
0,0.227286,0.292387,0.493584
1,0.959612,0.035790,0.021158
2,0.959800,0.039709,0.018284
3,0.959800,0.039709,0.018284
4,0.959612,0.035790,0.021158


Now that we trained the models and computed the probabilities in each origin. To classify each observation we want to select the origin with the highest probability of classification for that observation.

While each column in our dataframe (testing_probs) represents an origin we just need to choose the one with the largest probability. We can use the Dataframe method .idxmax() to return a Series where each value corresponds to the column or where the maximum value occurs for that observation. We need to make sure to set the axis paramater to 1 since we want to calculate the maximum value across columns. Since each column maps directly to an origin the resulting Series will be the classification from our model.

In [25]:
predicted_origins = testing_probs.idxmax(axis = 1)
predicted_origins.head()

0    3
1    1
2    1
3    1
4    1
dtype: int64

In this project, we learned the basics of extending logistic regression to work for multi-class classification problems.